In [ ]:
import wave
import gc
import random
import librosa
import librosa.display
import IPython
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.python.client import device_lib

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import multiprocessing as mp
import os
import cv2

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
/tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
def extract_sample(n_way,n_distractor, n_support,n_unlabel, n_query, datax, datay,dataunlabelx,dataunlabely):
  sample = []
  sample_unlabel=[]
  sample_distractor_unlabel=[]
  K = np.random.choice(np.unique(datay),n_way, replace = False)
  V = [x for x in np.unique(datay) if x not in np.unique(K)]
  D = np.random.choice(V,n_distractor)
  for cls in K:
    datax_cls = datax[datay == cls]
    datax_cls_unlabel=dataunlabelx[dataunlabely == cls]
    perm = np.random.permutation(datax_cls)
    permunlabel = np.random.permutation(datax_cls_unlabel)
    sample_cls = perm[:(n_support+n_query)]
    sample_cls_unlabel=permunlabel[:n_unlabel]
    sample_unlabel.append(sample_cls_unlabel)
    sample.append(sample_cls)
  for cls in D:
    datax_cls_unlabel=dataunlabelx[dataunlabely == cls]
    permunlabel = np.random.permutation(datax_cls_unlabel)
    sample_cls_unlabel=permunlabel[:n_unlabel]
    sample_distractor_unlabel.append(sample_cls_unlabel)
  sample = np.array(sample)
  #print(sample)
  sample_unlabel=np.array(sample_unlabel)
  sample_distractor_unlabel=np.array(sample_distractor_unlabel)
  sample = torch.from_numpy(sample).float()
  sample_unlabel=torch.from_numpy(sample_unlabel).float()
  sample_distractor_unlabel=torch.from_numpy(sample_distractor_unlabel).float()
  sample = sample.permute(0,1,4,2,3)
  sample_unlabel = sample_unlabel.permute(0,1,4,2,3)
  sample_distractor_unlabel=sample_distractor_unlabel.permute(0,1,4,2,3)
  return({
      'images': sample,
      'n_way': n_way,
      'n_distractor' : n_distractor,
      'n_support': n_support,
      'n_unlabel':n_unlabel,
      'n_query': n_query,
      'unlabelimages':sample_unlabel,
      'distractor_images':sample_distractor_unlabel
      })

In [ ]:
class Flatten(nn.Module):
  def __init__(self):
    super(Flatten, self).__init__()

  def forward(self, x):
    return x.view(x.size(0), -1)

def load_protonet_conv(**kwargs):
 
  x_dim = kwargs['x_dim']
  hid_dim = kwargs['hid_dim']
  z_dim = kwargs['z_dim']
  def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(2))
  encoder = nn.Sequential(
    conv_block(x_dim[0], hid_dim),
    conv_block(hid_dim, hid_dim),
    conv_block(hid_dim, hid_dim),
    conv_block(hid_dim, z_dim),
    Flatten())
  return ProtoNet(encoder)

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms


In [ ]:
def mlp(x):
  linear1 = nn.Linear(5, 20, bias=True).cuda()
  x = linear1(x)
  x = torch.tanh(x)
  linear2 = nn.Linear(20, 2, bias=True).cuda()
  x = linear2(x)
  return x.cuda()



In [ ]:
class ProtoNet(nn.Module):
  def __init__(self, encoder):
   
    super(ProtoNet, self).__init__()
    self.encoder = encoder.cuda()

  def set_forward_loss(self, sample):
   
    sample_images = sample['images'].cuda()
    sample_unlabel=sample['unlabelimages'].cuda()
    sample_distractor=sample['distractor_images'].cuda()
    n_way = sample['n_way']
    n_support = sample['n_support']
    n_query = sample['n_query']
    n_unlabel=sample['n_unlabel']
    x_support = sample_images[:, :n_support]
    x_query = sample_images[:, n_support:]
    x_unlabel=sample_unlabel
    x_distractor=sample_distractor
    target_inds = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, n_query, 1).long()
    target_inds = Variable(target_inds, requires_grad=False)
    target_inds = target_inds.cuda()
    ########################FOR UNLABELLED SET 
    xun = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_unlabel.contiguous().view(n_way * n_unlabel, *x_unlabel.size()[2:])], 0)
    zun = self.encoder.forward(xun)
    zun_dim = zun.size(-1)
    zun_protostar = zun[:n_way*n_support].view(n_way, n_support, zun_dim).mean(1)

    zun_protostar=torch.cat([zun_protostar, torch.zeros(5,4096).cuda()])
    xun_d = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_unlabel.contiguous().view(n_way * n_unlabel, *x_unlabel.size()[2:]), x_distractor.contiguous().view(n_way * n_unlabel, *x_distractor.size()[2:])],0)
    zun_d = self.encoder.forward(xun_d)
    #print(zun_d.shape)
    zun_d_dim = zun_d.size(-1)
    zun_protostarcopy=torch.clone(zun_protostar)

    #zun_protostarcopy=zun[:n_way*n_support].view(n_way, n_support, zun_dim).mean(1)
    #zun_unlabelquery = zun[n_way*n_unlabel:]   
    zun_unlabelquery = zun_d[n_way*n_support:]
    name=zun_unlabelquery.view(n_way+n_distractor, n_support, zun_d_dim) 
    distsstar=[]
    for i in range(n_way):
      distsstar = euclidean_dist(zun_unlabelquery, zun_protostar)+0.5*np.log(2*np.pi)
    for i in range(n_way, n_distractor):
      distsstar.append(0.0001*euclidean_dist(zun_unlabelquery, zun_protostar)+0.5*np.log(2*np.pi)+np.log(100)) 
    distsum = distsstar.sum(0)
    for i in range(len(distsum)):
      for j in range(distsstar.shape[0]):
        distsstar[j][i] = (distsstar[j][i] * distsstar.shape[0])/distsum[i]
    mean = distsstar.mean(0)
    skew = ((distsstar - mean / distsstar.std(0))**3).mean(0)
    kurt = ((distsstar - mean / distsstar.std(0))**4).mean(0)
    max = (distsstar.min(0))[0]
    min = (distsstar.min(0))[0]
    var = distsstar.var(0)
    #mlp_feat =torch.zeros(n_way,2).cuda()
    dist_features = torch.reshape(torch.cat([min, max, var, skew, kurt]),[-1, 5])
    mlp_feat = mlp(dist_features)
    #mlp_feat =torch.zeros(n_way,2).cuda()
    #print(mlp_feat[:, 1])
    #print(mlp_feat[0].shape)
    mic = torch.zeros(n_way*n_support,n_way+n_distractor).cuda()
  
    for i in range(n_way+n_distractor):
      for j in range(n_way*n_support):
        l = mlp_feat[i][1]*(-distsstar[j][i] + mlp_feat[i][0])
        mic[j][i] = torch.sigmoid(l)  
    micnew=mic.view(n_way, n_unlabel, -1)
    #log_p_ystar = F.log_softmax(-distsstar,dim=1).view(n_way, n_support, -1) 
    log_p_ystar = F.log_softmax(-distsstar,dim=1).view(n_way, n_unlabel, -1)
    loss_valstar = -log_p_ystar.gather(2, target_inds).squeeze().view(-1).mean()
    m = nn.Softmax(dim=1)
    outputnew = m(distsstar).view(n_way, n_unlabel, -1)
    _, y_hat = log_p_ystar.max(2)
    acc_valold1 = torch.eq(y_hat, target_inds.squeeze()).float().mean()
    prototype=losss(y_hat,x_unlabel,name,zun_d_dim,n_way,zun_protostarcopy,n_support,micnew,outputnew,n_unlabel)
    ##################FOR UNLABELLEDSET+SUPPORT SET PROTOTYPE FOUND---> now using new prototype for the calculation
    x = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_query.contiguous().view(n_way * n_query, *x_query.size()[2:])], 0)
    z=self.encoder.forward(x)
    z_query = z[n_way*n_support:] #for query set 
    ####FOR QUERY SET WITHOUT UNLABELED
    distold=euclidean_dist(z_query,zun_protostar)# old prototype
    #log_p_ystarold = F.log_softmax(-distold,dim=1).view(n_way, n_support, -1)
    log_p_ystarold = F.log_softmax(-distold,dim=1).view(n_way, n_query, -1)
    loss_valold = -log_p_ystarold.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hatold = log_p_ystarold.max(2)
    acc_valold = torch.eq(y_hatold, target_inds.squeeze()).float().mean()
    # print("accuracy on query set without adding unlabeled")
    # print(acc_valold)
    # print(prototype)
    #### END OF QUERY SET WITHOUT UNLABELED
    #### FOR QUERY SET WITH UNLABELED
    dists = euclidean_dist(z_query, prototype) # distance between query and new prototype
    log_p_ystar = F.log_softmax(-dists,dim=1).view(n_way, n_query, -1)
    loss_val = -log_p_ystar.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hatfinal = log_p_ystar.max(2)
    acc_val = torch.eq(y_hatfinal, target_inds.squeeze()).float().mean()
    # print(acc_val)
     #### FOR END OF QUERY SET WITH UNLABELED
    return loss_val, {
        'loss': loss_val.item(),
        'acc': acc_val.item(),
        'y_hat': y_hatfinal
        }

In [ ]:
#making of new prototype ---> From unlabeled set too
def losss(y_hat,x_unlabel,name,zdim,n_way,zun_protostarcopy,n_support,mic,outputnew,n_unlabel):
  ar=torch.zeros(n_way+n_distractor,zdim).cuda()
  j1=0
  sizearray=[]
  p=0
  for j1 in range(n_way+n_distractor):
    zun_protostarcopy[p]=zun_protostarcopy[p]*n_support
    sizearray.append(n_support)
    p=p+1
  j=0
  for k in y_hat:
    i=0
    for kk in k:
      for y in range(len(zun_protostarcopy)):
        zun_protostarcopy[y]=zun_protostarcopy[y]+name[j][i]*outputnew[j][i][y]*mic[j][i][y]
        sizearray[y]=sizearray[y]+outputnew[j][i][y]*mic[j][i][y]
      i=i+1
    j=j+1
  k=0
  for l in sizearray:
    if(l-int(l)>0.5):
      l=l+1
    zun_protostarcopy[k]=zun_protostarcopy[k]/int(l)
    k=k+1
  return zun_protostarcopy

In [ ]:

def euclidean_dist(x, y):
  n = x.size(0)
  m = y.size(0)
  d = x.size(1)
  assert d == y.size(1)

  x = x.unsqueeze(1).expand(n, m, d)
  y = y.unsqueeze(0).expand(n, m, d)

  return torch.pow(x - y, 2).sum(2)

In [ ]:
from tqdm import tqdm_notebook
from tqdm import tnrange

In [ ]:
def train(model, optimizer, train_x, train_y, n_way,n_distractor, n_support,n_unlabel, n_query, max_epoch, epoch_size,Xunlabel,Yunlabel):

  #divide the learning rate by 2 at each epoch, as suggested in paper
  scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
  epoch = 0 #epochs done so far
  stop = False #status to know when to stop

  while epoch < max_epoch and not stop:
    running_loss = 0.0
    running_acc = 0.0
    for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch+1)):
      sample = extract_sample(n_way, n_distractor, n_support,n_unlabel, n_query, train_x, train_y,Xunlabel,Yunlabel)
      optimizer.zero_grad()
      loss, output = model.set_forward_loss(sample)
      running_loss += output['loss']
      running_acc += output['acc']
      loss.backward()
      optimizer.step()
    epoch_loss = running_loss / epoch_size
    epoch_acc = running_acc / epoch_size
    print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch+1,epoch_loss, epoch_acc))
    epoch += 1
    scheduler.step()
    

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SECTION FOR TRAINING TESTING ON DATASET D1 URBANSOUND8K AND FSD DATASET



In [ ]:
Xtrain=np.load("/content/drive/MyDrive/urbansound8k/300data/final/xtrainlabel.npy")
Ytrain=np.load("/content/drive/MyDrive/urbansound8k/300data/final/ytrainlabel.npy")
Xunlabel=np.load("/content/drive/MyDrive/urbansound8k/300data/final/xtrainunlabel.npy")
Yunlabel=np.load("/content/drive/MyDrive/urbansound8k/300data/final/ytrainunlabel.npy")

In [ ]:
Xunlabeltest = np.load('/content/drive/MyDrive/urbansound8k/300data/final/xtestunlabel.npy')
Yunlabeltest = np.load('/content/drive/MyDrive/urbansound8k/300data/final/ytestunlabel.npy')
Xtest = np.load('/content/drive/MyDrive/urbansound8k/300data/final/xtestlabel.npy')
Ytest = np.load('/content/drive/MyDrive/urbansound8k/300data/final/ytestlabel.npy')

In [ ]:
%%time
import torch
from tensorflow import keras
model = load_protonet_conv(
    x_dim=(4,130,130),
    hid_dim=64,
    z_dim=64,
    )
optimizer = optim.Adam(model.parameters(), lr = 0.003)
n_way = 5
n_support = 5
n_query = 5
n_unlabel= 5
n_distractor=5
train_x = Xtrain
train_y = Ytrain
max_epoch = 10
epoch_size = 2000
train(model, optimizer, train_x, train_y, n_way,n_distractor, n_support, n_unlabel, n_query, max_epoch, epoch_size,Xunlabel,Yunlabel)
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  # This is added back by InteractiveShellApp.init_path()



Epoch 1 -- Loss: 1.2698 Acc: 0.7298



Epoch 2 -- Loss: 0.4447 Acc: 0.8261



Epoch 3 -- Loss: 0.3480 Acc: 0.8658



Epoch 4 -- Loss: 0.2655 Acc: 0.8991


...


Epoch 6 -- Loss: 0.1438 Acc: 0.9457



Epoch 7 -- Loss: 0.1095 Acc: 0.9593



Epoch 8 -- Loss: 0.0988 Acc: 0.9635



Epoch 9 -- Loss: 0.0906 Acc: 0.9663



Epoch 10 -- Loss: 0.0874 Acc: 0.9675
CPU times: user 2h 30min 33s, sys: 7min 52s, total: 2h 38min 26s
Wall time: 2h 39min 55s


In [ ]:
torch.save(model,"/content/drive/MyDrive/urbansound8k/300data/final/300data_3.1.3_distractor")

In [ ]:
model=torch.load("/content/drive/MyDrive/urbansound8k/300data/final/300data_3.1.3_distractor")

In [ ]:
def test(model, test_x, test_y, n_way,n_distractor, n_support,n_unlabel, n_query, test_episode,Xunlabel,Yunlabel):
  running_loss = 0.0
  running_acc = 0.0
  for episode in tnrange(test_episode):
    sample = extract_sample(n_way, n_distractor, n_support,n_unlabel, n_query, test_x, test_y,Xunlabel,Yunlabel)
    loss, output = model.set_forward_loss(sample)
    running_loss += output['loss']
    running_acc += output['acc']
  avg_loss = running_loss / test_episode
  avg_acc = running_acc / test_episode
  print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

In [ ]:
n_way = 5
n_support = 5
n_query = 5
n_unlabel=5
n_distractor=5
test_x = Xtest
test_y = Ytest
test_episode = 2000
test(model, test_x, test_y, n_way,n_distractor, n_support,n_unlabel, n_query, test_episode,Xunlabeltest,Yunlabeltest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.



Test results -- Loss: 0.8712 Acc: 0.6893


SECTION FOR TRAINING TESTING ON DATASET D1 URBANSOUND8K AND FSD DATASET
ENDS

SECTION FOR TRAINING TESTING ON DATASET D2 FSD DATASET ONLY



In [ ]:
Xtrain=np.load("/content/drive/MyDrive/urbansound8k/300data/xtrainlabelFSD.npy")
Ytrain=np.load("/content/drive/MyDrive/urbansound8k/300data/ytrainlabelFSD.npy")
Xunlabel=np.load("/content/drive/MyDrive/urbansound8k/300data/xtrainunlabelFSD.npy")
Yunlabel=np.load("/content/drive/MyDrive/urbansound8k/300data/ytrainunlabelFSD.npy")

In [ ]:
Xunlabeltest = np.load('/content/drive/MyDrive/urbansound8k/300data/xtestunlabelFSD.npy')
Yunlabeltest = np.load('/content/drive/MyDrive/urbansound8k/300data/ytestunlabelFSD.npy')
Xtest = np.load('/content/drive/MyDrive/urbansound8k/300data/xtestlabelFSD.npy')
Ytest = np.load('/content/drive/MyDrive/urbansound8k/300data/ytestlabelFSD.npy')

In [ ]:
%%time
import torch
from tensorflow import keras
model = load_protonet_conv(
    x_dim=(4,130,130),
    hid_dim=64,
    z_dim=64,
    )
optimizer = optim.Adam(model.parameters(), lr = 0.003)
n_way = 5
n_support = 5
n_query = 5
n_unlabel= 5
n_distractor=5
train_x = Xtrain
train_y = Ytrain
max_epoch = 10
epoch_size = 2000
train(model, optimizer, train_x, train_y, n_way,n_distractor, n_support, n_unlabel, n_query, max_epoch, epoch_size,Xunlabel,Yunlabel)
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  # This is added back by InteractiveShellApp.init_path()



Epoch 1 -- Loss: 1.7053 Acc: 0.6261



Epoch 2 -- Loss: 0.6511 Acc: 0.7572



Epoch 3 -- Loss: 0.4729 Acc: 0.8245



Epoch 4 -- Loss: 0.3160 Acc: 0.8828


...


Epoch 6 -- Loss: 0.1169 Acc: 0.9578



Epoch 7 -- Loss: 0.0816 Acc: 0.9710



Epoch 8 -- Loss: 0.0608 Acc: 0.9776



Epoch 9 -- Loss: 0.0572 Acc: 0.9792



Epoch 10 -- Loss: 0.0533 Acc: 0.9805
CPU times: user 2h 27min 21s, sys: 8min 10s, total: 2h 35min 31s
Wall time: 2h 36min 51s


In [ ]:
torch.save(model,"/content/drive/MyDrive/urbansound8k/300data/final/3.1.3FSD_distractor.pt")


In [ ]:
model=torch.load("/content/drive/MyDrive/urbansound8k/300data/final/3.1.3FSD_distractor.pt")

In [ ]:
def test(model, test_x, test_y, n_way, n_distractor, n_support, n_unlabel, n_query, test_episode, Xunlabel, Yunlabel):
  running_loss = 0.0
  running_acc = 0.0
  for episode in tnrange(test_episode):
    sample = extract_sample(n_way,n_distractor, n_support,n_unlabel, n_query, test_x, test_y,Xunlabel,Yunlabel)
    loss, output = model.set_forward_loss(sample)
    running_loss += output['loss']
    running_acc += output['acc']
  avg_loss = running_loss / test_episode
  avg_acc = running_acc / test_episode
  print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

In [ ]:
n_way = 5
n_support = 5
n_query = 5
n_unlabel=5
n_distractor=5
test_x = Xtest
test_y = Ytest
test_episode = 2000
test(model, test_x, test_y, n_way,n_distractor, n_support,n_unlabel, n_query, test_episode,Xunlabeltest,Yunlabeltest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.



Test results -- Loss: 2.0510 Acc: 0.4643


ENDS
